In [13]:
import json
import pandas as pd
from a2ginputstream import A2GClient
import random
import time
from datetime import datetime
from datetime import timedelta
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

## Read Data

In [14]:
USER_PAT, IKEY = "714091e2f8474eb9b60a7fd6429cfb2a", "467c80c4471743808fb1"
client = A2GClient(token=USER_PAT)

# {"start_on": {"$gte":start_hour}, "end_on":{"$lte":end_hour}}
movements_data:list[dict] = client.find(IKEY, {}, delete_id=True)

2024-11-25 13:55:46,083 - DEBUG - Starting new HTTPS connection (1): v2apigateway.a2g.io:443
2024-11-25 13:55:47,545 - DEBUG - https://v2apigateway.a2g.io:443 "GET /Inputstream/Ikey/467c80c4471743808fb1 HTTP/1.1" 200 None
2024-11-25 13:55:47,580 - INFO - Getting inputstream 467c80c4471743808fb1 from A2G...
2024-11-25 13:55:47,580 - INFO - Data - Ikey: 467c80c4471743808fb1, Checking cache expiration...
2024-11-25 13:55:47,585 - INFO - Data - Ikey: 467c80c4471743808fb1, Recovering data from cache...
2024-11-25 13:55:47,595 - INFO - Data - Ikey: 467c80c4471743808fb1, from cache


In [15]:
# USER_PAT, IKEY = "8f8ff571c7344ce9b6fae09f1ff9c5b1", "1c01a94bea5948faabda"
# client = A2GClient(token=USER_PAT)
time_now = datetime.utcnow().replace(
    microsecond=0,
    second=0,
    minute=0
)

# start_hour = time_now
# end_hour = time_now + timedelta(hours=1)

# # {"start_on": {"$gte":start_hour}, "end_on":{"$lte":end_hour}}
# movements_data:list[dict] = client.find(IKEY, {}, delete_id=True)

## Consolidate data by user and machine

In [16]:
df_movements = pd.DataFrame(movements_data)
df_users = df_movements.groupby(by="name")[["ton"]].sum()
df_machines = df_movements.groupby(by="machines")[["ton"]].sum()

df_machines.reset_index(inplace=True)
df_users.reset_index(inplace=True)

df_users["time"] = time_now.isoformat()
df_machines["time"] = time_now.isoformat()

In [17]:
df_machines

,machines,ton,time
0,C301,1464,2024-11-25T16:00:00
1,C302,908,2024-11-25T16:00:00
2,C303,618,2024-11-25T16:00:00


In [18]:
df_users

,name,ton,time
0,Ahbraham Smith,654,2024-11-25T16:00:00
1,Jane Doe,560,2024-11-25T16:00:00
2,John Doe,444,2024-11-25T16:00:00
3,John Smith,566,2024-11-25T16:00:00
4,Richard Alfredo,494,2024-11-25T16:00:00
5,Smith Alfred,272,2024-11-25T16:00:00
